In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Convertir las columnas categóricas a valores numéricos
# Usamos el código de categoría para convertir las columnas categóricas en valores numéricos
dataset['sex'] = dataset['sex'].astype('category').cat.codes
dataset['smoker'] = dataset['smoker'].astype('category').cat.codes
dataset['region'] = dataset['region'].astype('category').cat.codes

# Verificamos cómo quedó el dataset después de convertir las columnas
print("Dataset con columnas categóricas convertidas:\n", dataset.head())


In [ ]:
# Dividir el dataset en datos de entrenamiento y prueba
# Usamos un 80% de los datos para entrenamiento y un 20% para prueba
train_dataset, test_dataset = np.split(dataset, [int(.8 * len(dataset))])

# Imprimir el tamaño de los conjuntos de entrenamiento y prueba
print(f"Tamaño del conjunto de entrenamiento: {train_dataset.shape}")
print(f"Tamaño del conjunto de prueba: {test_dataset.shape}")


In [ ]:
# Separar las etiquetas (gastos) de los datos
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

# Verificamos que la columna 'expenses' haya sido eliminada correctamente
print("Primeras filas del conjunto de entrenamiento después de eliminar 'expenses':\n", train_dataset.head())


In [ ]:
# Normalizar los datos usando la capa de normalización de TensorFlow
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))

# Verificamos que la capa de normalización se haya creado correctamente
print("Capa de normalización adaptada con éxito.")


In [ ]:
# Mostrar la capa de normalización
normalizer


In [ ]:
# Crear el modelo de red neuronal
model = keras.Sequential([
    normalizer,
    layers.Dense(64, activation='relu'),  # Capa oculta con 64 neuronas y activación ReLU
    layers.Dense(32, activation='relu'),  # Capa oculta con 32 neuronas
    layers.Dense(1)  # Capa de salida con una neurona para predecir los gastos
])

# Compilar el modelo
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss='mae',  # Error absoluto medio
              metrics=['mae', 'mse'])

# Ver la estructura del modelo
model.summary()


In [ ]:
# Entrenar el modelo con los datos de entrenamiento
history = model.fit(train_dataset, train_labels, epochs=100, validation_split=0.2)

# Mostrar los resultados de la última época
print(f"Última época - Pérdida: {history.history['loss'][-1]}, MAE: {history.history['mae'][-1]}")


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
